In [40]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import lightgbm as lgb
%run Functions.ipynb
pd.set_option('display.max_columns', None)

In [7]:
perfect_sub = pd.read_csv("../Data/perfect_submission.csv")

In [2]:
train=pd.read_csv('train_feature_merged.csv')
train.head()

,id,target,hasNum,hasLink,hasUser,hasHashtag,exclamacion,interrogacion,mayusculas,longitud,...,Tiene_deaths,Tiene_climate,Tiene_plague,Tiene_disappearance,Tiene_missing,Tiene_floods,Tiene_delug,contains_keyword,Tiene_key_impor,Tiene_key_no_impor
0,1,1,0,0,0,1,0,0,10,71,...,0,0,0,0,0,0,0,0,0,0
1,4,1,0,0,0,0,0,0,5,40,...,0,0,0,0,0,0,0,0,0,0
2,5,1,0,0,0,0,0,0,2,135,...,0,0,0,0,0,0,0,0,0,0
3,6,1,1,0,0,1,0,0,1,67,...,0,0,0,0,0,0,0,0,0,0
4,7,1,0,0,0,1,0,0,3,90,...,0,0,0,0,0,0,0,0,0,0


In [3]:
to_predict=pd.read_csv('to_predict_feature_merged.csv')
to_predict.head()

,id,hasNum,hasLink,hasUser,hasHashtag,exclamacion,interrogacion,mayusculas,longitud,negativity_x,...,Tiene_deaths,Tiene_climate,Tiene_plague,Tiene_disappearance,Tiene_missing,Tiene_floods,Tiene_delug,contains_keyword,Tiene_key_impor,Tiene_key_no_impor
0,0,0,0,0,0,0,0,1,36,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,1,0,0,1,66,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,1,98,1,...,0,0,0,0,0,0,0,0,0,0
3,9,0,0,0,1,0,0,2,42,0,...,0,0,0,0,0,0,0,0,0,0
4,11,1,0,0,0,0,0,4,47,0,...,0,0,0,0,0,0,0,0,0,0


# DIVIDO LA INFO EN TRAIN Y TEST

In [11]:
x = train.drop(columns=['id','target'])
y = train['target']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25,random_state=42,shuffle=True)

print(f"Original shapes: X={x.shape} y={y.shape}")
print(f"Train shapes: X={x_train.shape} y={y_train.shape}")
print(f"Test  shapes: X={x_test.shape}  y={y_test.shape}")

Original shapes: X=(7613, 247) y=(7613,)
Train shapes: X=(5709, 247) y=(5709,)
Test  shapes: X=(1904, 247)  y=(1904,)


# PRUEBO EL MODELO CON TODOS CEROS O TODOS UNOS

In [15]:
todoCero= np.zeros(len(x_test))
print ("F1_score Test:", f1_score(y_test,todoCero,average='micro'))

F1_score Test: 0.5730042016806722


In [19]:
y_train.head()

5151    0
6351    0
3443    0
7164    1
7037    1
Name: target, dtype: int64

In [20]:
lgb_model = lgb.LGBMClassifier(boosting_type='dart', num_leaves=5, n_estimators=1000, metric='f1',
                               learning_rate=0.05, colsample_bytree=0.9)
lgb_model.fit(x_train, y_train)

LGBMClassifier(boosting_type='dart', class_weight=None, colsample_bytree=0.9,
               importance_type='split', learning_rate=0.05, max_depth=-1,
               metric='f1', min_child_samples=20, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=1000, n_jobs=-1, num_leaves=5,
               objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
               silent=True, subsample=1.0, subsample_for_bin=200000,
               subsample_freq=0)

In [26]:
predict_test=lgb_model.predict(x_test)
print ("F1_score Test:", f1_score(y_test,predict_test,average='micro'))

F1_score Test: 0.9732142857142857


In [35]:
prediction=lgb_model.predict(to_predict.drop('id',axis=1))
print ("F1_score Test:", f1_score(perfect_sub.target,prediction,average='micro'))

F1_score Test: 0.7486975176218204


In [41]:
result = submission_output(to_predict.id, prediction,"LigthGBMv1.csv")
result.head()

,id,target
0,0,1
1,2,1
2,3,0
3,9,0
4,11,1
